In [1]:
pip install spark-nlp==3.3.4

Note: you may need to restart the kernel to use updated packages.


In [45]:
!/opt/spark/bin/pyspark --packages com.audienceproject:spark-dynamodb_2.12:1.1.1,com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.4

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
[I 10:33:47.291 NotebookApp] The port 8888 is already in use, trying another port.
[I 10:33:47.291 NotebookApp] Serving notebooks from local directory: /spark-workspace/us-politicians-tweets/spark
[I 10:33:47.291 NotebookApp] Jupyter Notebook 6.4.6 is running at:
[I 10:33:47.291 NotebookApp] http://ip-172-31-87-148:8889/
[I 10:33:47.291 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
^C
[I 10:34:09.758 NotebookApp] interrupted
Serving notebooks from local directory: /spark-workspace/us-politicians-tweets/spark
0 active kernels
Jupyter Notebook 6.4.6 is running at:
http://ip-172-31-87-148:8889/
Shutdown this notebook server (y/[n])? resuming operation...


In [46]:
spark

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41975)
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:41975)

In [3]:
pip install numpy pandas nltk scikit-learn

Note: you may need to restart the kernel to use updated packages.


# Imports

In [4]:
from sparknlp.annotator import *
import pyspark.sql.functions as f
from pyspark.sql import Window
import pyspark.sql.types as t
from pyspark.ml.feature import Tokenizer as pysparkTokenizer, HashingTF, StopWordsRemover, CountVectorizer
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner, PerceptronModel, Chunker
from pyspark.ml.clustering import LDA
from nltk.corpus import stopwords
import pandas as pd
import nltk
import sparknlp
nltk.download('stopwords')

from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
print("Spark NLP version", sparknlp.version())

Spark NLP version 3.3.4


## Preprocess tweeter data

In [26]:
df_tweet = spark.read.option("tableName", "Tweets").format("dynamodb").load().select(f.col("text"))
df_tweet = df_tweet.withColumn("user", regexp_extract(col("text"), r"@([A-Za-z0-9]+)", 1))
df_tweet = (df_tweet.withColumn('text', regexp_replace('text', r"@[A-Za-z0-9]+", ''))
            .withColumn('text', regexp_replace('text', "RT[\s:]+", '')))
df_tweet.show()
f"Lenght: {df_tweet.count()}"

+--------------------+-------------+
|                text|         user|
+--------------------+-------------+
|It's refreshing t...|  TomCottonAR|
|We’re ending 2021...|        POTUS|
|The most popular ...|ChrisMurphyCT|
|Hearing the same ...|     RandPaul|
|Hearing the same ...|     RandPaul|
|We’re ending 2021...|        POTUS|
| https://t.co/ZT1...|        POTUS|
|The economy I inh...|        POTUS|
|Hearing the same ...|     RandPaul|
|      In your dreams|        POTUS|
|A Hillary Clinton...|      tedcruz|
|The most popular ...|ChrisMurphyCT|
|As we continue ou...|     JoeBiden|
|There is somethin...| amyklobuchar|
|When Harry Reid w...|  BarackObama|
| The analyst 👀 h...|        POTUS|
|Hearing the same ...|     RandPaul|
| be warning medic...|        POTUS|
| hemp no just jus...|        POTUS|
| if have that muc...|        POTUS|
+--------------------+-------------+
only showing top 20 rows



'Lenght: 1218'

In [27]:
df_acc = spark.read.option("header","true").csv("../accounts.csv").select("party", "username").distinct()
df_acc.show()
f"Lenght: {df_acc.count()}"

+-----+---------------+
|party|       username|
+-----+---------------+
|    D|  SenWhitehouse|
|    D| SenCoonsOffice|
|    R|     ThomTillis|
|    D|    SenStabenow|
|    R|RoundsforSenate|
|    D|       timkaine|
|    D|    SenJackReed|
|    R|  JohnnyIsakson|
|    D|      SenBooker|
|    R|  SenJohnHoeven|
|    R|  TheBushCenter|
|    R|RepublicanStudy|
|    R|SenatorLankford|
|    D| SenJeffMerkley|
|    D| MurrayCampaign|
|    R|   SenTomCotton|
|    R|     TeamCornyn|
|    R|    RogerWicker|
|    D|      Bob_Casey|
|    R|    TomCottonAR|
+-----+---------------+
only showing top 20 rows



'Lenght: 198'

In [28]:
df_tweet = df_tweet.join(df_acc, df_tweet.user == df_acc.username).select('party', 'text')
df_tweet.show()

+-----+--------------------+
|party|                text|
+-----+--------------------+
|    R|It's refreshing t...|
|    D|We’re ending 2021...|
|    D|The most popular ...|
|    R|Hearing the same ...|
|    R|Hearing the same ...|
|    D|We’re ending 2021...|
|    D| https://t.co/ZT1...|
|    D|The economy I inh...|
|    R|Hearing the same ...|
|    D|      In your dreams|
|    R|A Hillary Clinton...|
|    D|The most popular ...|
|    D|As we continue ou...|
|    D|There is somethin...|
|    D|When Harry Reid w...|
|    D| The analyst 👀 h...|
|    R|Hearing the same ...|
|    D| be warning medic...|
|    D| hemp no just jus...|
|    D| if have that muc...|
+-----+--------------------+
only showing top 20 rows



In [9]:
f"Lenght: {df_tweet.count()}"

'Lenght: 918'

## Preprocess reddit data

In [11]:
df_reddit = spark.read.option("tableName", "RedditPosts").format("dynamodb").load().select(f.col("submission_id").alias("text"), f.col("subreddit"))
df_reddit = df_reddit.withColumn("party", initcap(col('subreddit')).substr(1,1)).select("party", "text")
df_reddit.show()
f"Lenght: {df_reddit.count()}"

AnalysisException: cannot resolve '`submission_id`' given input columns: [];
'Project ['submission_id AS text#58, 'subreddit]
+- RelationV2[] RedditPosts


## Join both sources

In [29]:
df = df_tweet#.limit(1000)#.union(df_reddit)
df.show()
f"Lenght: {df.count()}"

+-----+--------------------+
|party|                text|
+-----+--------------------+
|    R|It's refreshing t...|
|    D|We’re ending 2021...|
|    D|The most popular ...|
|    R|Hearing the same ...|
|    R|Hearing the same ...|
|    D|We’re ending 2021...|
|    D| https://t.co/ZT1...|
|    D|The economy I inh...|
|    R|Hearing the same ...|
|    D|      In your dreams|
|    R|A Hillary Clinton...|
|    D|The most popular ...|
|    D|As we continue ou...|
|    D|There is somethin...|
|    D|When Harry Reid w...|
|    D| The analyst 👀 h...|
|    R|Hearing the same ...|
|    D| be warning medic...|
|    D| hemp no just jus...|
|    D| if have that muc...|
+-----+--------------------+
only showing top 20 rows



'Lenght: 1219'

## Train test dataset

In [30]:
df_g = df.groupBy("party")\
        .count()\
        .withColumn("share", col('count')/df.count())
df_g.show()

+-----+-----+------------------+
|party|count|             share|
+-----+-----+------------------+
|    D|  884|0.7222222222222222|
|    R|  347|0.2834967320261438|
+-----+-----+------------------+



In [31]:
train, test = df.randomSplit([0.7, 0.3], seed=42)

In [32]:
train.groupBy("party")\
    .count()\
    .withColumn("share", col('count')/train.count())\
    .show()

+-----+-----+------------------+
|party|count|             share|
+-----+-----+------------------+
|    D|  633|0.7250859106529209|
|    R|  249|0.2852233676975945|
+-----+-----+------------------+



In [33]:
test.groupBy("party")\
    .count()\
    .withColumn("share", col('count')/test.count())\
    .show()

+-----+-----+------------------+
|party|count|             share|
+-----+-----+------------------+
|    D|  261|  0.71900826446281|
|    R|  102|0.2809917355371901|
+-----+-----+------------------+



In [34]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

In [16]:
# embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L2_128") \
#       .setInputCols("document") \
#       .setOutputCol("sentence_embeddings")

In [17]:
embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L8_512") \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]


In [37]:
# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
    .setInputCols (["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("party")\
    .setMaxEpochs (20)\
    .setEnableOutputLogs (True) \
    .setLr(5*10**(-3)) \
    .setDropout(0.5)

In [38]:
bert_sent_clf_pipeline = Pipeline(
    stages = [
        document,
        embeddings,
        classsifierdl
    ])

In [40]:
bert_Model = bert_sent_clf_pipeline.fit(train)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41975)
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:41975)

In [ ]:
preds = bert_Model.transform(test)

In [ ]:
pandpred = preds.select("party","class.result").toPandas()
pandpred.result = pandpred.result.apply(lambda x: x[0])

In [23]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(classification_report(pandpred.party, pandpred.result))

              precision    recall  f1-score   support

           D       0.75      1.00      0.85       191
           R       0.00      0.00      0.00        65

    accuracy                           0.75       256
   macro avg       0.37      0.50      0.43       256
weighted avg       0.56      0.75      0.64       256



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
print(accuracy_score(pandpred.party, pandpred.result))

0.74609375


In [25]:
print(confusion_matrix(pandpred.party, pandpred.result))

[[191   0]
 [ 65   0]]
